# Recommender Systems

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

In [0]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv', inferSchema=True, header=True)

In [0]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



# Process Data

In [0]:
train, test = data.randomSplit([0.8, 0.2])

# Model

In [0]:
from pyspark.ml.recommendation import ALS

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [0]:
model = als.fit(train)

In [0]:
predictions = model.transform(test)

In [0]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      3|   1.0|    28|  1.6885886|
|      2|   1.0|    12|  1.2322564|
|      4|   1.0|    12| -0.9507862|
|      3|   1.0|     1| 0.99204063|
|      1|   1.0|     6|  3.8850508|
|      5|   1.0|     6| 0.17400654|
|      1|   1.0|     3|-0.10894406|
|      4|   2.0|    20|-0.34430984|
|      2|   1.0|    19|-0.09666571|
|      5|   2.0|    15|  0.8336924|
|      2|   3.0|     9| 0.23834184|
|      0|   1.0|     8|  1.8283098|
|      2|   4.0|     8| 0.74843484|
|      2|   2.0|     7|  3.1472018|
|      2|   1.0|    25| -1.6229491|
|      4|   1.0|    29| -0.5986641|
|      0|   1.0|    21|  1.4029081|
|      2|   4.0|    21| 0.93429375|
|      0|   1.0|    11| -0.5495827|
|      4|   3.0|     2|  3.5621352|
+-------+------+------+-----------+
only showing top 20 rows



# Evaluate

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [0]:
rmse = evaluator.evaluate(predictions)

In [0]:
print('RMSE')
print(rmse)

RMSE
1.724427268255428


# Use on a New User

In [0]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])

In [0]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     11|    11|
|     13|    11|
|     25|    11|
|     35|    11|
|     37|    11|
|     39|    11|
|     40|    11|
|     61|    11|
|     62|    11|
|     66|    11|
|     69|    11|
|     76|    11|
|     81|    11|
|     82|    11|
|     94|    11|
|     99|    11|
+-------+------+



In [0]:
recommendations = model.transform(single_user)

In [0]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     81|    11| 1.7464151|
|     39|    11| 1.4719008|
|     76|    11| 1.4713786|
|     11|    11| 1.2916837|
|     82|    11|  1.090697|
|     69|    11| 1.0155258|
|     13|    11| 1.0087812|
|     99|    11|0.72215486|
|     35|    11| 0.6605159|
|     66|    11|0.62924516|
|     25|    11|  0.605205|
|     61|    11|0.49752244|
|     40|    11|0.44403237|
|     37|    11|0.21668366|
|     62|    11|0.19849679|
|      0|    11|-0.5495827|
|     94|    11|-0.8030566|
+-------+------+----------+

